In [47]:
import pandas as pd
import os
import glob

def clean_all_clams_data(directory_path):
    '''Reformats all CLAMS data files (.csv) in the provided directory by dropping unnecessary rows.
    
    Parameters:
    directory_path (string): directory containing .csv files to clean
    
    Returns:
    Nothing. Prints new filenames saved to "Cleaned_CLAMS_data" directory.
    '''
    
    def clean_file(file_path, output_directory):
        '''Helper function to clean individual file.'''
        # Read the file as plain text to extract metadata
        with open(file_path, 'r') as f:
            lines = f.readlines()

        # Extract the "Subject ID" value
        for line in lines:
            if 'Subject ID' in line:
                subject_id = line.split(',')[1].strip()
                break

        # Read the data chunk of the CSV file
        df = pd.read_csv(file_path, skiprows=range(0, 22))

        # Drop additional 2 formatting rows
        df.drop([0, 1], inplace=True)

        # Construct the new file name
        base_name, ext = os.path.splitext(os.path.basename(file_path))
        new_file_name = f"{base_name}_ID{subject_id}{ext}"

        # Save the cleaned data to the new directory
        output_path = os.path.join(output_directory, new_file_name)
        df.to_csv(output_path, index=False)
        print(f"Cleaned CLAMS data saved to {output_path}")

    # Create the output directory if it doesn't exist
    output_directory = os.path.join(directory_path, "Cleaned_CLAMS_data")
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Process all CSV files in the directory
    all_files = glob.glob(os.path.join(directory_path, "*.CSV"))
    for file_path in all_files:
        clean_file(file_path, output_directory)

# Example usage
directory_path = "/home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs"
clean_all_clams_data(directory_path)

Cleaned CLAMS data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-06_PEN56_Pio_Trial_1.0105_ID2365.CSV
Cleaned CLAMS data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-06_PEN56_Pio_Trial_1.0103_ID2366.CSV
Cleaned CLAMS data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-06_PEN56_Pio_Trial_1.0101_ID2361.CSV
Cleaned CLAMS data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-06_PEN56_Pio_Trial_1.0107_ID3029.CSV
Cleaned CLAMS data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-11.0103_ID2370.CSV
Cleaned CLAMS data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-11.0104_ID2362.CSV
Cleaned CLAMS data saved to /home/pistillilab/

In [91]:
import pandas as pd
import os
from datetime import datetime, timedelta

def trim_all_clams_data(directory_path, trim_hours, keep_hours):
    '''Trims all cleaned CLAMS data files in the specified directory.
    
    Parameters:
    directory_path (string): path to the directory containing cleaned .csv files
    trim_hours (int): number of hours to trim from the beginning
    keep_hours (int): number of hours to keep in the resulting file
    
    Returns:
    Nothing. Saves the trimmed data to new CSV files in the "Trimmed_CLAMS_data" directory.
    '''
    
    # Create a new directory for trimmed files if it doesn't exist
    trimmed_directory = os.path.join(directory_path, "Trimmed_CLAMS_data")
    if not os.path.exists(trimmed_directory):
        os.makedirs(trimmed_directory)
    
    # List all files in the directory
    files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f)) and f.endswith('.CSV')]
    
    for file in files:
        file_path = os.path.join(directory_path, file)
        
        # Read the cleaned CSV file
        df = pd.read_csv(file_path)
        
        # Convert the 'DATE/TIME' column to datetime format
        df['DATE/TIME'] = pd.to_datetime(df['DATE/TIME'], errors='coerce')
        
        # Calculate the starting timestamp after trimming
        start_time = df['DATE/TIME'].iloc[0] + timedelta(hours=trim_hours)
        
        # Filter the dataframe to start from the trimmed timestamp
        df_trimmed = df[df['DATE/TIME'] >= start_time]
        
        # Note the value in the "LED LIGHTNESS" column after trimming
        initial_led_value = df_trimmed['LED LIGHTNESS'].iloc[0]
        
        # Find the index of the next change in the "LED LIGHTNESS" value
        led_lightness_change_index = df_trimmed[df_trimmed['LED LIGHTNESS'] != initial_led_value].index[0]
        
        # Calculate the ending timestamp
        end_time = df['DATE/TIME'].iloc[led_lightness_change_index] + timedelta(hours=keep_hours)
        
        # Filter the dataframe to start from the LED change and end at the specified timestamp
        df_result = df[(df['DATE/TIME'] >= df['DATE/TIME'].iloc[led_lightness_change_index]) & (df['DATE/TIME'] <= end_time)]
        
        # Save the resulting data to a new CSV file in the "Trimmed_CLAMS_data" directory
        base_name, ext = os.path.splitext(file)
        new_file_name = os.path.join(trimmed_directory, f"{base_name}_trimmed{ext}")
        df_result.to_csv(new_file_name, index=False)
        print(f"Trimmed data saved to {new_file_name}")

# Example usage
directory_path = "/home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data"
trim_all_clams_data(directory_path, trim_hours=24, keep_hours=72)

Trimmed data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-06_PEN56_Pio_Trial_1.0104_ID2371_trimmed.CSV
Trimmed data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-06_PEN56_Pio_Trial_1.0103_ID2366_trimmed.CSV
Trimmed data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-06_PEN56_Pio_Trial_1.0107_ID3029_trimmed.CSV
Trimmed data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-11.0103_ID2370_trimmed.CSV
Trimmed data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-06_PEN56_Pio_Trial_1.0102_ID2363_trimmed.CSV
Trimmed data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-06_PEN56_Pio_Trial_1.0101_ID2361_trimmed.CSV
Trimmed data sav

In [92]:
import pandas as pd
import numpy as np
from datetime import timedelta
import os

def bin_clams_data(file_path, bin_hours):
    df = pd.read_csv(file_path)
    
    # Convert 'DATE/TIME' column to datetime format
    df['DATE/TIME'] = pd.to_datetime(df['DATE/TIME'])
    
    # Drop unnecessary columns
    columns_to_drop = ["STATUS1", "O2IN", "O2OUT", "DO2", "CO2IN", "CO2OUT", "DCO2", "XTOT", "YTOT", "LED HUE", "LED SATURATION", "BIN"]
    df = df.drop(columns=columns_to_drop, errors='ignore')

    # Add TOT_AMB column to the original dataframe
    df['TOT_AMB'] = df['XAMB'] + df['YAMB']
    
    # Create a new column for bin labels
    df['BIN'] = np.nan
    
    # For each unique "LED LIGHTNESS" value, assign bin labels
    for led_value in df['LED LIGHTNESS'].unique():
        subset = df[df['LED LIGHTNESS'] == led_value].copy()
        start_time = subset['DATE/TIME'].iloc[0]
        bin_label = 0
        bin_labels = []
        
        for timestamp in subset['DATE/TIME']:
            if (timestamp - start_time) >= timedelta(hours=bin_hours):
                bin_label += 1
                start_time = timestamp
            bin_labels.append(bin_label)
        
        df.loc[subset.index, 'BIN'] = bin_labels
    
    # Columns to retain the last value in the bin
    last_val_columns = ["INTERVAL", "CHAN", "DATE/TIME", "ACCO2", "ACCCO2", "FEED1 ACC", "WHEEL ACC"]
    
    # Columns to sum within the bin
    sum_columns = ["WHEEL", "FEED1", "TOT_AMB"]
    
    # Columns to average (excluding the ones we're taking the last value or summing)
    avg_columns = df.columns.difference(last_val_columns + sum_columns + ['BIN', 'LED LIGHTNESS'])
    
    # Group by "LED LIGHTNESS" and "BIN" and calculate the mean, sum, or last value as appropriate
    df_binned = df.groupby(['LED LIGHTNESS', 'BIN']).agg({**{col: 'last' for col in last_val_columns},
                                                         **{col: 'mean' for col in avg_columns},
                                                         **{col: 'sum' for col in sum_columns}}).reset_index()
    
    # Add start and end time columns
    start_times = df.groupby(['LED LIGHTNESS', 'BIN'])['DATE/TIME'].first().reset_index(name='DATE/TIME_start')
    end_times = df.groupby(['LED LIGHTNESS', 'BIN'])['DATE/TIME'].last().reset_index(name='DATE/TIME_end')
    df_binned = pd.merge(df_binned, start_times, on=['LED LIGHTNESS', 'BIN'])
    df_binned = pd.merge(df_binned, end_times, on=['LED LIGHTNESS', 'BIN'])
    
    # Add start and end interval columns
    start_intervals = df.groupby(['LED LIGHTNESS', 'BIN'])['INTERVAL'].first().reset_index(name='INTERVAL_start')
    end_intervals = df.groupby(['LED LIGHTNESS', 'BIN'])['INTERVAL'].last().reset_index(name='INTERVAL_end')
    df_binned = pd.merge(df_binned, start_intervals, on=['LED LIGHTNESS', 'BIN'])
    df_binned = pd.merge(df_binned, end_intervals, on=['LED LIGHTNESS', 'BIN'])
    
    # Calculate the duration of each bin in hours
    df_binned['DURATION'] = (df_binned['DATE/TIME_end'] - df_binned['DATE/TIME_start']).dt.total_seconds() / 3600
    
    # Remove the BIN column and sort based on INTERVAL_start
    df_binned = df_binned.drop(columns=['BIN']).sort_values(by='INTERVAL_start')
    
    # Reorder columns based on your request
    desired_order = ["CHAN", "INTERVAL_start", "INTERVAL_end", "DATE/TIME_start", "DATE/TIME_end", "DURATION", 
                     "VO2", "ACCO2", "VCO2", "ACCCO2", "RER", "HEAT", "FLOW", "PRESSURE", "FEED1", "FEED1 ACC", 
                     "TOT_AMB", "WHEEL", "WHEEL ACC", "ENCLOSURE TEMP", "ENCLOSURE SETPOINT", "LED LIGHTNESS"]
    df_binned = df_binned[desired_order]
    
    # Round all variables to 4 decimal places
    df_binned = df_binned.round(4)
    
    # Save the binned data to a new CSV file
    output_path = file_path.replace("Trimmed_CLAMS_data", "Binned_CLAMS_data").replace(".CSV", "_binned.CSV")
    
    # Check if the directory exists, if not, create it
    output_directory = os.path.dirname(output_path)
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    df_binned.to_csv(output_path, index=False)

# Example usage
file_path = "/home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/Trimmed_CLAMS_data/2023-07-06_PEN56_Pio_Trial_1.0101_ID2361_trimmed.CSV"
bin_clams_data(file_path, bin_hours=)


SyntaxError: invalid syntax (1594415279.py, line 90)

In [93]:
import pandas as pd
import numpy as np
from datetime import timedelta
import os

def bin_clams_data(file_path, bin_hours):
    # ... [rest of your bin_clams_data function remains unchanged]

def process_directory(directory_path, bin_hours):
    # Get a list of all .CSV files in the directory
    csv_files = [f for f in os.listdir(directory_path) if f.endswith('.CSV') and os.path.isfile(os.path.join(directory_path, f))]
    
    # Process each .CSV file
    for csv_file in csv_files:
        file_path = os.path.join(directory_path, csv_file)
        bin_clams_data(file_path, bin_hours)
        print(f"Processed {csv_file}")

# Example usage
directory_path = "/home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/Trimmed_CLAMS_data"
process_directory(directory_path, bin_hours=12)


FileNotFoundError: [Errno 2] No such file or directory: '/home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/Trimmed_CLAMS_data'

In [96]:
import pandas as pd
import os
import glob

def main_process_clams_data(directory_path, trim_hours, keep_hours, bin_hours):
    '''Main function to process all CLAMS data files in the provided directory.
    
    Parameters:
    directory_path (string): directory containing .csv files to process
    trim_hours (int): number of hours to trim from the beginning of the cleaned data
    keep_hours (int): number of hours to keep in the trimmed data
    bin_hours (int): number of hours to bin the data
    
    Returns:
    Nothing. Prints progress and saves processed files to respective directories.
    '''
    
    print("Cleaning all CLAMS data...")
    clean_all_clams_data(directory_path)
    
    print("\nTrimming all cleaned CLAMS data...")
    cleaned_data_directory = os.path.join(directory_path, "Cleaned_CLAMS_data")
    trim_all_clams_data(cleaned_data_directory, trim_hours, keep_hours)
    
    print("\nBinning all trimmed CLAMS data...")
    trimmed_data_directory = os.path.join(directory_path, "Cleaned_CLAMS_data", "Trimmed_CLAMS_data")
    process_directory(trimmed_data_directory, bin_hours)

# Example usage
directory_path = "/home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs"
main_process_clams_data(directory_path, trim_hours=24, keep_hours=72, bin_hours=4)

Cleaning all CLAMS data...
Cleaned CLAMS data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-06_PEN56_Pio_Trial_1.0105_ID2365.CSV
Cleaned CLAMS data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-06_PEN56_Pio_Trial_1.0103_ID2366.CSV
Cleaned CLAMS data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-06_PEN56_Pio_Trial_1.0101_ID2361.CSV
Cleaned CLAMS data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-06_PEN56_Pio_Trial_1.0107_ID3029.CSV
Cleaned CLAMS data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-11.0103_ID2370.CSV
Cleaned CLAMS data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-11.0104_ID2362.CSV
Cleaned CLAMS data 

FileNotFoundError: [Errno 2] No such file or directory: '/home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/Trimmed_CLAMS_data'

In [100]:
import pandas as pd
import os
import glob
import numpy as np
from datetime import datetime, timedelta

def clean_all_clams_data(directory_path):
    '''Reformats all CLAMS data files (.csv) in the provided directory by dropping unnecessary rows.
    
    Parameters:
    directory_path (string): directory containing .csv files to clean
    
    Returns:
    Nothing. Prints new filenames saved to "Cleaned_CLAMS_data" directory.
    '''
    
    def clean_file(file_path, output_directory):
        '''Helper function to clean individual file.'''
        # Read the file as plain text to extract metadata
        with open(file_path, 'r') as f:
            lines = f.readlines()

        # Extract the "Subject ID" value
        for line in lines:
            if 'Subject ID' in line:
                subject_id = line.split(',')[1].strip()
                break

        # Read the data chunk of the CSV file
        df = pd.read_csv(file_path, skiprows=range(0, 22))

        # Drop additional 2 formatting rows
        df.drop([0, 1], inplace=True)

        # Construct the new file name
        base_name, ext = os.path.splitext(os.path.basename(file_path))
        new_file_name = f"{base_name}_ID{subject_id}{ext}"

        # Save the cleaned data to the new directory
        output_path = os.path.join(output_directory, new_file_name)
        df.to_csv(output_path, index=False)
        print(f"Cleaned CLAMS data saved to {output_path}")

    # Create the output directory if it doesn't exist
    output_directory = os.path.join(directory_path, "Cleaned_CLAMS_data")
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Process all CSV files in the directory
    all_files = glob.glob(os.path.join(directory_path, "*.CSV"))
    for file_path in all_files:
        clean_file(file_path, output_directory)

def trim_all_clams_data(directory_path, trim_hours, keep_hours):
    '''Trims all cleaned CLAMS data files in the specified directory.
    
    Parameters:
    directory_path (string): path to the directory containing cleaned .csv files
    trim_hours (int): number of hours to trim from the beginning
    keep_hours (int): number of hours to keep in the resulting filefiles = [f for f in os.listdir(cleaned_directory) 
    
    Returns:
    Nothing. Saves the trimmed data to new CSV files in the "Trimmed_CLAMS_data" directory.
    '''
    
    # Create a new directory for trimmed files if it doesn't exist
    trimmed_directory = os.path.join(directory_path, "Trimmed_CLAMS_data")
    if not os.path.exists(trimmed_directory):
        os.makedirs(trimmed_directory)
    
    # List all files in the directory
    files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f)) and f.endswith('.CSV')]
    
    for file in files:
        file_path = os.path.join(directory_path, file)
        
        # Read the cleaned CSV file
        df = pd.read_csv(file_path)
        
        # Convert the 'DATE/TIME' column to datetime format
        df['DATE/TIME'] = pd.to_datetime(df['DATE/TIME'], errors='coerce')
        
        # Calculate the starting timestamp after trimming
        start_time = df['DATE/TIME'].iloc[0] + timedelta(hours=trim_hours)
        
        # Filter the dataframe to start from the trimmed timestamp
        df_trimmed = df[df['DATE/TIME'] >= start_time]
        
        # Note the value in the "LED LIGHTNESS" column after trimming
        initial_led_value = df_trimmed['LED LIGHTNESS'].iloc[0]
        
        # Find the index of the next change in the "LED LIGHTNESS" value
        led_lightness_change_index = df_trimmed[df_trimmed['LED LIGHTNESS'] != initial_led_value].index[0]
        
        # Calculate the ending timestamp
        end_time = df['DATE/TIME'].iloc[led_lightness_change_index] + timedelta(hours=keep_hours)
        
        # Filter the dataframe to start from the LED change and end at the specified timestamp
        df_result = df[(df['DATE/TIME'] >= df['DATE/TIME'].iloc[led_lightness_change_index]) & (df['DATE/TIME'] <= end_time)]
        
        # Save the resulting data to a new CSV file in the "Trimmed_CLAMS_data" directory
        base_name, ext = os.path.splitext(file)
        new_file_name = os.path.join(trimmed_directory, f"{base_name}_trimmed{ext}")
        df_result.to_csv(new_file_name, index=False)
        print(f"Trimmed data saved to {new_file_name}")

def bin_clams_data(file_path, bin_hours):
    df = pd.read_csv(file_path)
    
    # Convert 'DATE/TIME' column to datetime format
    df['DATE/TIME'] = pd.to_datetime(df['DATE/TIME'])
    
    # Drop unnecessary columns
    columns_to_drop = ["STATUS1", "O2IN", "O2OUT", "DO2", "CO2IN", "CO2OUT", "DCO2", "XTOT", "YTOT", "LED HUE", "LED SATURATION", "BIN"]
    df = df.drop(columns=columns_to_drop, errors='ignore')

    # Add TOT_AMB column to the original dataframe
    df['TOT_AMB'] = df['XAMB'] + df['YAMB']
    
    # Create a new column for bin labels
    df['BIN'] = np.nan
    
    # For each unique "LED LIGHTNESS" value, assign bin labels
    for led_value in df['LED LIGHTNESS'].unique():
        subset = df[df['LED LIGHTNESS'] == led_value].copy()
        start_time = subset['DATE/TIME'].iloc[0]
        bin_label = 0
        bin_labels = []
        
        for timestamp in subset['DATE/TIME']:
            if (timestamp - start_time) >= timedelta(hours=bin_hours):
                bin_label += 1
                start_time = timestamp
            bin_labels.append(bin_label)
        
        df.loc[subset.index, 'BIN'] = bin_labels
    
    # Columns to retain the last value in the bin
    last_val_columns = ["INTERVAL", "CHAN", "DATE/TIME", "ACCO2", "ACCCO2", "FEED1 ACC", "WHEEL ACC"]
    
    # Columns to sum within the bin
    sum_columns = ["WHEEL", "FEED1", "TOT_AMB"]
    
    # Columns to average (excluding the ones we're taking the last value or summing)
    avg_columns = df.columns.difference(last_val_columns + sum_columns + ['BIN', 'LED LIGHTNESS'])
    
    # Group by "LED LIGHTNESS" and "BIN" and calculate the mean, sum, or last value as appropriate
    df_binned = df.groupby(['LED LIGHTNESS', 'BIN']).agg({**{col: 'last' for col in last_val_columns},
                                                         **{col: 'mean' for col in avg_columns},
                                                         **{col: 'sum' for col in sum_columns}}).reset_index()
    
    # Add start and end time columns
    start_times = df.groupby(['LED LIGHTNESS', 'BIN'])['DATE/TIME'].first().reset_index(name='DATE/TIME_start')
    end_times = df.groupby(['LED LIGHTNESS', 'BIN'])['DATE/TIME'].last().reset_index(name='DATE/TIME_end')
    df_binned = pd.merge(df_binned, start_times, on=['LED LIGHTNESS', 'BIN'])
    df_binned = pd.merge(df_binned, end_times, on=['LED LIGHTNESS', 'BIN'])
    
    # Add start and end interval columns
    start_intervals = df.groupby(['LED LIGHTNESS', 'BIN'])['INTERVAL'].first().reset_index(name='INTERVAL_start')
    end_intervals = df.groupby(['LED LIGHTNESS', 'BIN'])['INTERVAL'].last().reset_index(name='INTERVAL_end')
    df_binned = pd.merge(df_binned, start_intervals, on=['LED LIGHTNESS', 'BIN'])
    df_binned = pd.merge(df_binned, end_intervals, on=['LED LIGHTNESS', 'BIN'])
    
    # Calculate the duration of each bin in hours
    df_binned['DURATION'] = (df_binned['DATE/TIME_end'] - df_binned['DATE/TIME_start']).dt.total_seconds() / 3600
    
    # Remove the BIN column and sort based on INTERVAL_start
    df_binned = df_binned.drop(columns=['BIN']).sort_values(by='INTERVAL_start')
    
    # Reorder columns based on your request
    desired_order = ["CHAN", "INTERVAL_start", "INTERVAL_end", "DATE/TIME_start", "DATE/TIME_end", "DURATION", 
                     "VO2", "ACCO2", "VCO2", "ACCCO2", "RER", "HEAT", "FLOW", "PRESSURE", "FEED1", "FEED1 ACC", 
                     "TOT_AMB", "WHEEL", "WHEEL ACC", "ENCLOSURE TEMP", "ENCLOSURE SETPOINT", "LED LIGHTNESS"]
    df_binned = df_binned[desired_order]
    
    # Round all variables to 4 decimal places
    df_binned = df_binned.round(4)
    
    # Save the binned data to a new CSV file
    output_path = file_path.replace("Trimmed_CLAMS_data", "Binned_CLAMS_data").replace(".CSV", "_binned.CSV")
    
    # Check if the directory exists, if not, create it
    output_directory = os.path.dirname(output_path)
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    df_binned.to_csv(output_path, index=False)

def process_directory(directory_path, bin_hours):
    # Get a list of all .CSV files in the directory
    csv_files = [f for f in os.listdir(directory_path) if f.endswith('.CSV') and os.path.isfile(os.path.join(directory_path, f))]
    
    # Process each .CSV file
    for csv_file in csv_files:
        file_path = os.path.join(directory_path, csv_file)
        bin_clams_data(file_path, bin_hours)
        print(f"Processed {csv_file}")

def main_process_clams_data(directory_path, trim_hours, keep_hours, bin_hours):
    '''Main function to process all CLAMS data files in the provided directory.
    
    Parameters:
    directory_path (string): directory containing .csv files to process
    trim_hours (int): number of hours to trim from the beginning of the cleaned data
    keep_hours (int): number of hours to keep in the trimmed data
    bin_hours (int): number of hours to bin the data
    
    Returns:
    Nothing. Prints progress and saves processed files to respective directories.
    '''
    
    print("Cleaning all CLAMS data...")
    clean_all_clams_data(directory_path)
    
    print("\nTrimming all cleaned CLAMS data...")
    cleaned_data_directory = os.path.join(directory_path, "Cleaned_CLAMS_data")
    trim_all_clams_data(cleaned_data_directory, trim_hours, keep_hours)
    
    print("\nBinning all trimmed CLAMS data...")
    trimmed_data_directory = os.path.join(directory_path, "Cleaned_CLAMS_data", "Trimmed_CLAMS_data")
    process_directory(trimmed_data_directory, bin_hours)

# Example usage
directory_path = "/home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs"
main_process_clams_data(directory_path, trim_hours=24, keep_hours=72, bin_hours=4)

Cleaning all CLAMS data...
Cleaned CLAMS data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-06_PEN56_Pio_Trial_1.0105_ID2365.CSV
Cleaned CLAMS data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-06_PEN56_Pio_Trial_1.0103_ID2366.CSV
Cleaned CLAMS data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-06_PEN56_Pio_Trial_1.0101_ID2361.CSV
Cleaned CLAMS data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-06_PEN56_Pio_Trial_1.0107_ID3029.CSV
Cleaned CLAMS data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-11.0103_ID2370.CSV
Cleaned CLAMS data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-11.0104_ID2362.CSV
Cleaned CLAMS data 

In [103]:
import pandas as pd
import os
import glob
import numpy as np
from datetime import datetime, timedelta

def clean_all_clams_data(directory_path):
    '''Reformats all CLAMS data files (.csv) in the provided directory by dropping unnecessary rows.
    
    Parameters:
    directory_path (string): directory containing .csv files to clean
    
    Returns:
    Nothing. Prints new filenames saved to "Cleaned_CLAMS_data" directory.
    '''
    
    def clean_file(file_path, output_directory):
        '''Helper function to clean individual file.'''
        # Read the file as plain text to extract metadata
        with open(file_path, 'r') as f:
            lines = f.readlines()

        # Extract the "Subject ID" value
        for line in lines:
            if 'Subject ID' in line:
                subject_id = line.split(',')[1].strip()
                break

        # Read the data chunk of the CSV file
        df = pd.read_csv(file_path, skiprows=range(0, 22))

        # Drop additional 2 formatting rows
        df.drop([0, 1], inplace=True)

        # Construct the new file name
        base_name, ext = os.path.splitext(os.path.basename(file_path))
        new_file_name = f"{base_name}_ID{subject_id}{ext}"

        # Save the cleaned data to the new directory
        output_path = os.path.join(output_directory, new_file_name)
        df.to_csv(output_path, index=False)
        print(f"Cleaned CLAMS data saved to {output_path}")

    # Create the output directory if it doesn't exist
    output_directory = os.path.join(directory_path, "Cleaned_CLAMS_data")
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Process all CSV files in the directory
    all_files = glob.glob(os.path.join(directory_path, "*.CSV"))
    for file_path in all_files:
        clean_file(file_path, output_directory)

def trim_all_clams_data(directory_path, trim_hours, keep_hours):
    '''Trims all cleaned CLAMS data files in the specified directory.
    
    Parameters:
    directory_path (string): path to the directory containing cleaned .csv files
    trim_hours (int): number of hours to trim from the beginning
    keep_hours (int): number of hours to keep in the resulting file
    
    Returns:
    Nothing. Saves the trimmed data to new CSV files in the "Trimmed_CLAMS_data" directory.
    '''
    
    # Create a new directory for trimmed files if it doesn't exist
    trimmed_directory = os.path.join(directory_path, "Trimmed_CLAMS_data")
    if not os.path.exists(trimmed_directory):
        os.makedirs(trimmed_directory)

    # Get the path to the cleaned data files
    cleaned_directory = os.path.join(directory_path, "Cleaned_CLAMS_data")
    
    # List all files in the directory
    files = [f for f in os.listdir(cleaned_directory) if os.path.isfile(os.path.join(cleaned_directory, f)) and f.endswith('.CSV')]
    
    for file in files:
        file_path = os.path.join(cleaned_directory, file)
        
        # Read the cleaned CSV file
        df = pd.read_csv(file_path)
        
        # Convert the 'DATE/TIME' column to datetime format
        df['DATE/TIME'] = pd.to_datetime(df['DATE/TIME'], errors='coerce')
        
        # Calculate the starting timestamp after trimming
        start_time = df['DATE/TIME'].iloc[0] + timedelta(hours=trim_hours)
        
        # Filter the dataframe to start from the trimmed timestamp
        df_trimmed = df[df['DATE/TIME'] >= start_time]
        
        # Note the value in the "LED LIGHTNESS" column after trimming
        initial_led_value = df_trimmed['LED LIGHTNESS'].iloc[0]
        
        # Find the index of the next change in the "LED LIGHTNESS" value
        led_lightness_change_index = df_trimmed[df_trimmed['LED LIGHTNESS'] != initial_led_value].index[0]
        
        # Calculate the ending timestamp
        end_time = df['DATE/TIME'].iloc[led_lightness_change_index] + timedelta(hours=keep_hours)
        
        # Filter the dataframe to start from the LED change and end at the specified timestamp
        df_result = df[(df['DATE/TIME'] >= df['DATE/TIME'].iloc[led_lightness_change_index]) & (df['DATE/TIME'] <= end_time)]
        
        # Save the resulting data to a new CSV file in the "Trimmed_CLAMS_data" directory
        base_name, ext = os.path.splitext(file)
        new_file_name = os.path.join(trimmed_directory, f"{base_name}_trimmed{ext}")
        df_result.to_csv(new_file_name, index=False)
        print(f"Trimmed data saved to {new_file_name}")

def bin_clams_data(file_path, bin_hours):
    df = pd.read_csv(file_path)
    
    # Convert 'DATE/TIME' column to datetime format
    df['DATE/TIME'] = pd.to_datetime(df['DATE/TIME'])
    
    # Drop unnecessary columns
    columns_to_drop = ["STATUS1", "O2IN", "O2OUT", "DO2", "CO2IN", "CO2OUT", "DCO2", "XTOT", "YTOT", "LED HUE", "LED SATURATION", "BIN"]
    df = df.drop(columns=columns_to_drop, errors='ignore')

    # Add TOT_AMB column to the original dataframe
    df['TOT_AMB'] = df['XAMB'] + df['YAMB']
    
    # Create a new column for bin labels
    df['BIN'] = np.nan
    
    # For each unique "LED LIGHTNESS" value, assign bin labels
    for led_value in df['LED LIGHTNESS'].unique():
        subset = df[df['LED LIGHTNESS'] == led_value].copy()
        start_time = subset['DATE/TIME'].iloc[0]
        bin_label = 0
        bin_labels = []
        
        for timestamp in subset['DATE/TIME']:
            if (timestamp - start_time) >= timedelta(hours=bin_hours):
                bin_label += 1
                start_time = timestamp
            bin_labels.append(bin_label)
        
        df.loc[subset.index, 'BIN'] = bin_labels
    
    # Columns to retain the last value in the bin
    last_val_columns = ["INTERVAL", "CHAN", "DATE/TIME", "ACCO2", "ACCCO2", "FEED1 ACC", "WHEEL ACC"]
    
    # Columns to sum within the bin
    sum_columns = ["WHEEL", "FEED1", "TOT_AMB"]
    
    # Columns to average (excluding the ones we're taking the last value or summing)
    avg_columns = df.columns.difference(last_val_columns + sum_columns + ['BIN', 'LED LIGHTNESS'])
    
    # Group by "LED LIGHTNESS" and "BIN" and calculate the mean, sum, or last value as appropriate
    df_binned = df.groupby(['LED LIGHTNESS', 'BIN']).agg({**{col: 'last' for col in last_val_columns},
                                                         **{col: 'mean' for col in avg_columns},
                                                         **{col: 'sum' for col in sum_columns}}).reset_index()
    
    # Add start and end time columns
    start_times = df.groupby(['LED LIGHTNESS', 'BIN'])['DATE/TIME'].first().reset_index(name='DATE/TIME_start')
    end_times = df.groupby(['LED LIGHTNESS', 'BIN'])['DATE/TIME'].last().reset_index(name='DATE/TIME_end')
    df_binned = pd.merge(df_binned, start_times, on=['LED LIGHTNESS', 'BIN'])
    df_binned = pd.merge(df_binned, end_times, on=['LED LIGHTNESS', 'BIN'])
    
    # Add start and end interval columns
    start_intervals = df.groupby(['LED LIGHTNESS', 'BIN'])['INTERVAL'].first().reset_index(name='INTERVAL_start')
    end_intervals = df.groupby(['LED LIGHTNESS', 'BIN'])['INTERVAL'].last().reset_index(name='INTERVAL_end')
    df_binned = pd.merge(df_binned, start_intervals, on=['LED LIGHTNESS', 'BIN'])
    df_binned = pd.merge(df_binned, end_intervals, on=['LED LIGHTNESS', 'BIN'])
    
    # Calculate the duration of each bin in hours
    df_binned['DURATION'] = (df_binned['DATE/TIME_end'] - df_binned['DATE/TIME_start']).dt.total_seconds() / 3600
    
    # Remove the BIN column and sort based on INTERVAL_start
    df_binned = df_binned.drop(columns=['BIN']).sort_values(by='INTERVAL_start')
    
    # Reorder columns based on your request
    desired_order = ["CHAN", "INTERVAL_start", "INTERVAL_end", "DATE/TIME_start", "DATE/TIME_end", "DURATION", 
                     "VO2", "ACCO2", "VCO2", "ACCCO2", "RER", "HEAT", "FLOW", "PRESSURE", "FEED1", "FEED1 ACC", 
                     "TOT_AMB", "WHEEL", "WHEEL ACC", "ENCLOSURE TEMP", "ENCLOSURE SETPOINT", "LED LIGHTNESS"]
    df_binned = df_binned[desired_order]
    
    # Round all variables to 4 decimal places
    df_binned = df_binned.round(4)
    
    # Save the binned data to a new CSV file
    output_path = file_path.replace("Trimmed_CLAMS_data", "Binned_CLAMS_data").replace(".CSV", "_binned.CSV")
    
    # Check if the directory exists, if not, create it
    output_directory = os.path.dirname(output_path)
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    df_binned.to_csv(output_path, index=False)

def process_directory(directory_path, bin_hours):

    # Get path to trimmed directory
    trimmed_directory = os.path.join(directory_path, "Trimmed_CLAMS_data")
    
    # Get a list of all .CSV files in the directory
    csv_files = [f for f in os.listdir(trimmed_directory) if f.endswith('.CSV') and os.path.isfile(os.path.join(trimmed_directory, f))]
    
    # Process each .CSV file
    for csv_file in csv_files:
        file_path = os.path.join(trimmed_directory, csv_file)
        bin_clams_data(file_path, bin_hours)
        print(f"Processed {csv_file}")

def main_process_clams_data(directory_path, trim_hours, keep_hours, bin_hours):
    '''Main function to process all CLAMS data files in the provided directory.
    
    Parameters:
    directory_path (string): directory containing .csv files to process
    trim_hours (int): number of hours to trim from the beginning of the cleaned data
    keep_hours (int): number of hours to keep in the trimmed data
    bin_hours (int): number of hours to bin the data
    
    Returns:
    Nothing. Prints progress and saves processed files to respective directories.
    '''
    
    print("Cleaning all CLAMS data...")
    clean_all_clams_data(directory_path)
    
    print("\nTrimming all cleaned CLAMS data...")
    trim_all_clams_data(directory_path, trim_hours, keep_hours)
    
    print("\nBinning all trimmed CLAMS data...")
    process_directory(directory_path, bin_hours)

# Example usage
directory_path = "/home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs"
main_process_clams_data(directory_path, trim_hours=24, keep_hours=72, bin_hours=4)

Cleaning all CLAMS data...
Cleaned CLAMS data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-06_PEN56_Pio_Trial_1.0105_ID2365.CSV
Cleaned CLAMS data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-06_PEN56_Pio_Trial_1.0103_ID2366.CSV
Cleaned CLAMS data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-06_PEN56_Pio_Trial_1.0101_ID2361.CSV
Cleaned CLAMS data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-06_PEN56_Pio_Trial_1.0107_ID3029.CSV
Cleaned CLAMS data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-11.0103_ID2370.CSV
Cleaned CLAMS data saved to /home/pistillilab/Documents/PistilliLab/Stuart Clayton/All_subjects_CSVs/Cleaned_CLAMS_data/2023-07-11.0104_ID2362.CSV
Cleaned CLAMS data 